In [1]:
from keras.models import load_model
import mtcnn
import cv2
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
# load the model
model = load_model('facenet_keras.h5',compile = True)

Using TensorFlow backend.
C:\Users\rushi\anaconda3\envs\python364\lib\site-packages\keras\engine\saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [2]:
def face_detection(pix,detector):
    results = detector.detect_faces(pix)
    if results:
        x1, y1, width, height = results[0]['box']
        x1, y1 = abs(x1), abs(y1)
        x2, y2 = x1 + width, y1 + height
        face = pix1[y1:y2, x1:x2]
        face = cv2.resize(face,(160,160))
        return face
    return None

In [3]:
doc1 = {"Name":[] , "Count":[]}
path = 'C:\\Users\\rushi\\Documents\\CV_project\\Intro_DB\\'
name_list = os.listdir(path)
for name in name_list:
    doc1["Name"].append(name)
    new_path = path + name + '\\'
    n = os.listdir(new_path)
    doc1["Count"].append(len(n))
    
print(doc1)

{'Name': ['amey', 'anamika', 'gauri', 'piyush', 'richa', 'riddhi'], 'Count': [22, 23, 24, 27, 21, 22]}


In [4]:
doc2 = {"label":[],"images":[]}
path = "Intro_DB\\"
detector = mtcnn.MTCNN()
for length in range(len(doc1["Name"])):
    name = doc1["Name"][length]
    c = doc1["Count"][length]
    new_path = path + name + "\\" + name
    for i in range(c):
        new_path1 = new_path + str(i+1) + ".jpg"
        img1 = cv2.imread(new_path1)
        pix1 = np.asarray(img1)
        face = face_detection(pix1,detector)
        if type(face) != type(None): 
            doc2['label'].append(name)
            doc2['images'].append(face)


In [5]:
print(doc2["label"])

['amey', 'amey', 'amey', 'amey', 'amey', 'amey', 'amey', 'amey', 'amey', 'amey', 'amey', 'amey', 'amey', 'amey', 'amey', 'amey', 'amey', 'amey', 'anamika', 'anamika', 'anamika', 'anamika', 'anamika', 'anamika', 'anamika', 'anamika', 'anamika', 'anamika', 'anamika', 'anamika', 'anamika', 'anamika', 'anamika', 'anamika', 'anamika', 'anamika', 'anamika', 'anamika', 'gauri', 'gauri', 'gauri', 'gauri', 'gauri', 'gauri', 'gauri', 'gauri', 'gauri', 'gauri', 'gauri', 'gauri', 'gauri', 'gauri', 'gauri', 'gauri', 'gauri', 'gauri', 'gauri', 'gauri', 'gauri', 'gauri', 'gauri', 'piyush', 'piyush', 'piyush', 'piyush', 'piyush', 'piyush', 'piyush', 'piyush', 'piyush', 'piyush', 'piyush', 'piyush', 'piyush', 'piyush', 'piyush', 'piyush', 'piyush', 'piyush', 'piyush', 'piyush', 'piyush', 'piyush', 'piyush', 'piyush', 'richa', 'richa', 'richa', 'richa', 'richa', 'richa', 'richa', 'richa', 'richa', 'richa', 'richa', 'richa', 'richa', 'richa', 'richa', 'richa', 'richa', 'richa', 'riddhi', 'riddhi', 'riddh

In [6]:
label = np.array(doc2["label"])
images = np.array(doc2["images"])

trainX, testX, trainy, testy = train_test_split(
images, label, test_size=0.9, random_state=42)
            

In [7]:
# get the face embedding for one face
def get_embedding(model, face_pixels):
    # scale pixel values
    face_pixels = face_pixels.astype('float32')
    # standardize pixel values across channels (global)
    mean, std = face_pixels.mean(), face_pixels.std()
    face_pixels = (face_pixels - mean) / std
    # transform face into one sample
    samples = np.expand_dims(face_pixels, axis=0)
    # make prediction to get embedding
    yhat = model.predict(samples)
    return yhat[0]

In [8]:
# convert each face in the train set to an embedding
newTrainX = list()
for face_pixels in trainX:
    embedding = get_embedding(model, face_pixels)
    newTrainX.append(embedding)
newTrainX = np.asarray(newTrainX)
print(newTrainX.shape)
# convert each face in the test set to an embedding
newTestX = list()
for face_pixels in testX:
    embedding = get_embedding(model, face_pixels)
    newTestX.append(embedding)
newTestX = np.asarray(newTestX)
print(newTestX.shape)

(12, 128)
(111, 128)


In [9]:
# normalize input vectors
in_encoder = Normalizer(norm='l2')
trainX = in_encoder.transform(newTrainX)
testX = in_encoder.transform(newTestX)
# label encode targets
out_encoder = LabelEncoder()
out_encoder.fit(trainy)
trainy = out_encoder.transform(trainy)
testy = out_encoder.transform(testy)
# fit model
model = SVC(kernel='linear', probability=True)
model.fit(trainX, trainy)
# predict
yhat_train = model.predict(trainX)
yhat_test = model.predict(testX)
# score
score_train = accuracy_score(trainy, yhat_train)
score_test = accuracy_score(testy, yhat_test)
# summarize
print('Accuracy: train=%.3f, test=%.3f' % (score_train*100, score_test*100))



Accuracy: train=100.000, test=67.568
